<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Bias-variance lab

---

In this lab you will explore how bias and variance change using a dataset on college statistics.

In [1]:
import numpy as np
import scipy 
import seaborn as sns
import pandas as pd
import patsy

from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.model_selection import cross_val_score, cross_val_predict,train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.style.use('fivethirtyeight')

np.random.seed(1)

## Load the data

In [2]:
college = pd.read_csv('../../../../resource-datasets/college_stats/College.csv')
college.rename(columns={'Unnamed: 0':'College'}, inplace=True)
college.head(3)

,College,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54


### Check the names of the variables and the data types contained in the college data

### Clean the column names (replace "." by "_" and transform to lower case)

### Transform the variable "Private" into 1s and 0s rather than yes/no

### Choose "grad_rate" as target variable

### Create a feature matrix containing all variables except "grad_rate" and the college names

### Use the standard scaler to rescale your features and response

Hint: to rescale the response variable, you will first have to bring it intro 2D-array form and later to retransform it into 1D-array form.

In [3]:
from sklearn.preprocessing import StandardScaler

### Fit a 10-fold cross-validated linear regression model for grad_rate using all other features. Evaluate the model performance using cross_val_score, obtain the r2_score and mean_squared_error for each fold and averaged over all folds.

### Create a function with the name `predict_from_samples` that will iteratively predict your target from different train-test splits

This will be used to calculate the bias and the variance afterwards.
We want to produce a range of different models fitted on a random choice of data points and making predictions for the remaining data points.


Your function should:

1. take the following arguments:
    * `model`: a regression model 
    * `X`: predictor matrix/dataframe 
    * `y`: target variable 
    * `number_of_splits`: a number indicating how many times the dataset should be split randomly into training and test sets

2. return a dataframe `yhat_tracker` containing columns for 
    * the true values of `y` (obtained from the `y` passed as an argument)
    * the predictions made for y in each of the `number_of_splits` into training/test sets

3. in the function body
    - initialize the dataframe `yhat_tracker` with a single column `y` for the true values
    - initialize a list `rowinds` containing the indices of yhat_tracker
    - create a loop over `number_of_splits`
        - within the loop, create a train/test split of rowinds
        - create training and test sets from y and X by subsetting on the indices obtained from the train/test split
        - fit the model on the training data
        - obtain predictions for those y which are currently in the test set
        - set the predicted values for those `y` which are currently in the training set to `np.nan` 
        - insert the predictions from the current loop as a new column into `yhat_tracker`
    - return yhat_tracker 
    
In the end, the returned data frame should contain one column with the true y-values and `number_of_splits` columns with predicted y-values for the different test sets. Each of the prediction columns contains only as many values as there have been in the test set each time.

In [4]:
def predict_from_samples(model, X, y, number_of_splits=100):
    
    yhat_tracker = pd.DataFrame({'ytrue':y})
    
    rowinds = list(range(X.shape[0]))
    
    for i in range(number_of_splits):
        
        train_inds, test_inds = train_test_split(rowinds, test_size=0.33)
                
        X_train, Y_train = X.iloc[train_inds, :], y[train_inds]
        X_test, Y_test = X.iloc[test_inds, :], y[test_inds]
        
        model.fit(X_train, Y_train)
        yhats = model.predict(X_test)
        
        yhat_tracker['sample'+str(i+1)] = np.nan
        yhat_tracker.iloc[test_inds, -1] = yhats
        
    return yhat_tracker

### Create different predictor datasets

To see what happens to bias and variance as the predictors change, create a few versions of X that have different numbers of predictors in them:

* model $X$ from above including all features
* a model $X_{small}$ including only one feature (e.g. personal)
* a model $X_{overfit}$ created with the patsy formula below 
    - taking the cube takes into account 
        - all original features
        - all features that could be created by squaring or cubing a single column
        - all products of two or three different columns
        - all products of the squared of one column with a different column
        - -1 excludes the intercept)

In [5]:
#overfit_formula = '~ ('+' + '.join(X.columns)+')**3 -1'
#X_overfit = patsy.dmatrix(overfit_formula, data=X, return_type='dataframe')

In [6]:
#X_small = X[['personal']]

### Use the `predict_from_samples` function you wrote above to get the predicted values for $X$, $X_{small}$, $X_{overfit}$ 

- Run each of your X through the function with the y target vector. 
- Recall that the output of your function has the true values of y in one column and then predicted values of y for the different train-test splits in other columns

### Calculate bias and variance 

Below are two functions to calculate bias and variance from the dataframe returned by your function `predict_from_samples`.
You can use them to calculate the bias and variance for the models containing different feature combinations. 

If you have more predictors, variance of prediction should generally go up and bias go down. Likewise, if you have few predictors variance should go down and bias go up.

For a bad model, they both might go up a lot!

In [7]:
def calculate_bias_sq(yhats_df):
    # Take out the true values of y that are in the first column:
    ytrue = yhats_df.iloc[:,0]
    
    # Calculate the mean of the predictions, averaged across the columns.
    # So, all of the predictions for the true y at row 0 would be averaged together
    # and so on for all the rows.
    yhat_means = yhats_df.iloc[:,1:].mean(axis=1)
    
    # Subtract the true value of y from the mean of the predicted values, and square it.
    elementwise_bias_sq = (yhat_means - ytrue)**2
    
    # Take the mean of those squared bias values (across all y)
    mean_bias_sq = np.mean(elementwise_bias_sq)

    return mean_bias_sq

def calculate_variance(yhats_df):
    
    # Calculate the mean of the predicted y's across the columns (mean of yhat for each row)
    yhats_means = yhats_df.iloc[:,1:].mean(axis=1)
    
    # subtract the mean of the yhats from the original yhat values (for each row)
    # and square the result. 
    yhats_devsq = (yhats_df.iloc[:,1:].subtract(yhats_means, axis=0))**2
    
    # Take the mean of the squared deviations from the mean, then 
    # take the mean of those to get the overall variance across the y observations
    yhats_devsq_means = yhats_devsq.mean(axis=1)
    
    return np.mean(yhats_devsq_means)

### How does regularization affect bias and variance?

Use lasso and/or ridge regression on your dataset with all the predictor variables. In your function `predict_from_samples` replace `model` with your choice.

In [8]:
from sklearn.linear_model import Lasso, Ridge

lasso = Lasso(alpha=2.0)
ridge = Ridge(alpha=2.1)